In [1]:
%load_ext cython

## Creating extension types

An extension type is a python class that has definitions in Cython. A dict and a list has Cython equivalent class. Since Cython cannot be debugged, it does not need to have a `__dict__` obj or the overhead associated with it.

In [2]:
%%cython

cdef class Particle:
    cdef double mass, position, velocity
    
    def __init__(self, mass, position, velocity):
        self.mass = mass
        self.position = position
        self.velocity = velocity
    
    cpdef getMass(self):
        return self.mass
    
    def __add__(self, other):
        return self.getMass() + other.getMass()

In [3]:
Particle(1,2,3) + Particle(1,2,3)

2.0

In [4]:
try:
    Particle(1,2,3) + None
except:
    print('Adding to the wrong type does not work.')

Adding to the wrong type does not work.


Bear in mind that this class is private and anything defined as cdef cannot be directly accessed by python. The `__init__` function has access but the `__add__` function does not.

In [5]:
Particle(1,2,3)

## Creating constant variables in a Cython class

In [6]:
%%cython

cdef class PrivateParticle:
    cdef readonly double mass, position, velocity
    
    def __init__(self, mass, position, velocity):
        self.mass = mass
        self.position = position
        self.velocity = velocity
    
    cpdef destroyMass(self):
        self.mass = 0


In [7]:
particle = PrivateParticle(1,2,3)

In [8]:
particle.destroyMass()

In [9]:
particle.mass

0.0

In [10]:
try:
    particle.mass = 3
except:
    print('Cannot change value from python but can view it')

Cannot change value from python but can view it


In [11]:
%%cython

cdef class PublicParticle:
    cdef public double mass, position, velocity
    
    def __init__(self, mass, position, velocity):
        self.mass = mass
        self.position = position
        self.velocity = velocity

In [12]:
public_particle = PublicParticle(1,2,3)

Python can do whatever it likes with public attributes.

In [13]:
public_particle.position += 1

## Initialize a structure from C

In [14]:
%%cython

cdef class SmallImage:
    cdef:
        unsigned char n_rows, n_cols
    
    def _cinit_(self, unsigned char n_rows, unsigned char n_cols):
        self.n_rows = n_rows
        self.n_cols = n_cols

cdef class SmallImagePython:
    cdef:
        unsigned char n_rows, n_cols
    
    def _init_(self, unsigned char n_rows, unsigned char n_cols):
        self.n_rows = n_rows
        self.n_cols = n_cols

In [15]:
small_img = SmallImage(100, 100)

In [16]:
%timeit SmallImage(100, 100)

154 ns ± 21 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [17]:
%timeit SmallImagePython(100, 100)

134 ns ± 7.29 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [18]:
%%cython

from cpython.mem cimport PyMem_Malloc, PyMem_Realloc, PyMem_Free

cdef class SmallImageData:
    cdef:
        unsigned int n_rows, n_cols
        double* _matrix

    def _cinit_(self, unsigned int n_rows, unsigned int n_cols):
        self.n_rows  = n_rows
        self.n_cols  = n_cols
        cdef int mem_size = n_rows * n_cols * sizeof(double)
        self._matrix = <double*>PyMem_Malloc(mem_size)

        if self._matrix == NULL:
            raise MemoryError()
    
    def __dealloc__(self):
        if self._matrix != NULL:
            PyMem_Free(self._matrix)

import numpy as np
            
cdef class PySmallImageData:
    cdef:
        unsigned int n_rows, n_cols

    def _init_(self, unsigned int n_rows, unsigned int n_cols):
        self.n_rows  = n_rows
        self.n_cols  = n_cols
        self._matrix = np.zeros((n_rows, n_cols), dtype=np.float64)

It is actually faster using numpy to allocate a matrix but for non-standard data structure.

In [19]:
%timeit SmallImageData(10000, 10000)

147 ns ± 9.73 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [20]:
%timeit PySmallImageData(10000, 10000)

127 ns ± 3.98 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


## Subclasses

Subclasses can be declared in cells as well put looking up a class from another class can be expensive.

In [21]:
%%cython 

cdef class SuperClass:
    cdef double velocity, mass

    def __cinit__(self, velocity, mass):
        self.velocity = velocity
        self.mass     = mass

    cpdef getMomentum(self):
        return self.velocity * self.mass

cpdef dispatch(obj):
    (<SuperClass>obj).getMomentum()

def dispatch2(obj not None):
    (<SuperClass>obj).getMomentum()
    
cdef class MidClass(SuperClass):
    pass
    
cdef class SubClass(MidClass):

    cpdef getEnergy(self):
        return 0.5 * self.mass * self.velocity**2

    cpdef getMoment(self):
        return self.mass * self.velocity

Sometimes we may dispatch a method to a base class as though it were the superclass. We can use casting or <i>polymorphism</i>. In this case, the presence of a middle class was enough to make dispatch viable.

In [22]:
sub_class = SubClass(1,2)

In [23]:
%timeit sub_class.getMoment()

84.1 ns ± 13.4 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [24]:
%timeit sub_class.getMomentum()

82 ns ± 7.38 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [25]:
%timeit dispatch(sub_class)

95.5 ns ± 14.7 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


## None type safe

The function call below when uncommented will crash the kernel. There is no proper checking involved and None is considered NULL in C++ which is an empty type. We can use dispatch2 in this case which will throw an error and stop the kernel from blowing up.

In [26]:
# dispatch(None)

In [27]:
try:
    dispatch2(None)
except:
    print('None type caught')

None type caught


This adds some overhead again.

In [28]:
%timeit dispatch2(sub_class)

87.8 ns ± 14.1 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


## Using a property from Cython

In [29]:
%%cython 

cdef class ClassWithProperty:
    cdef double velocity, _mass

    def __cinit__(self, double velocity, double mass):
        self.velocity = velocity
        self._mass     = mass

    property momentum:
        def __get__(self):
            return self.mass * self.velocity
        
        def __set__(self, val):
            self.velocity = val/self.mass

    property mass:
        def __get__(self):
            return self._mass
        
        def __set__(self, val):
            self._mass = val

In [30]:
prop = ClassWithProperty(1,2)

In [31]:
prop.momentum

2.0

## Overloading the add operator

The add operator in python has the <i>iadd</i> operator (inplace addition) to add something else to the class.

In [86]:
class Thing:
    def __init__(self, value : float):
        self.value = value
    
    def __add__(self, other):
        return self.value + other.value
    
    def __iadd__(self, other):
        self.value += other.value
        return self

In [87]:
thing1 = Thing(1)
thing2 = Thing(2)
thing1 += thing2

In [88]:
thing1 + thing2

5

In Cython, you have to overload the add function to do this. In this case, overloading is much simpler.

In [89]:
%%cython

cdef class Addable:
    cdef double value
    def __cinit__(self, double value):
        self.value = value
        
    def __add__(x, y):
        if isinstance(x, Addable) and isinstance(y, float):
            return (<Addable>x).value + y
        elif isinstance(y, Addable) and isinstance(x, float):
            return (<Addable>y).value + x
        elif isinstance(x, Addable) and isinstance(y, Addable):
            return (<Addable>y).value + (<Addable>x).value
        else:
            return NotImplemented

In [90]:
addy1 = Addable(1)
addy2 = Addable(2)

Cython is pretty forgiving when an integer is supplied instead of a float.

In [91]:
addy1 + addy2 + 4

7.0

In [92]:
try:
    addy1 + []
except:
    print('The addable class does not know about lists.')

The addable class does not know about lists.


## Comparison operations

The enum does not seem to be available. There is a way of making comparisons in cython instead of using all the magic functions.

In [107]:
%%cython

cdef class Comparison:
    cdef readonly double value
    def __cinit__(self, double value):
        self.value = value
    
    def __richcmp__(x, y, int op):
        cdef double x_val = (<Comparison>x).value if isinstance(x, Comparison) else x
        cdef double y_val = (<Comparison>y).value if isinstance(y, Comparison) else y
        
        if op == 0:
            return x_val < y_val
        elif op == 1:
            return x_val <= y_val
        elif op == 2:
            return x_val == y_val
        elif op == 3:
            return x_val != y_val
        elif op == 4:
            return x_val > y_val
        elif op == 5:
            return x_val >= y_val
        else:
            assert False

In [108]:
c1 = Comparison(1)
c2 = Comparison(2)

In [109]:
c1 < c2

True

In [110]:
c1 <= c2

True

In [111]:
c1 == c2

False

In [112]:
c1 != c2

True

In [113]:
c1 > c2

False

In [114]:
c1 >= c2

False

In [115]:
d1 = {'value': 1}
d2 = {'value': 2}

In [116]:
%timeit c1 >= c2

81.5 ns ± 5.51 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [117]:
%timeit c1 < c2

89 ns ± 9.22 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [119]:
%timeit d1['value'] < d2['value']

179 ns ± 31.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [120]:
%timeit d1['value'] >= d2['value']

173 ns ± 25.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## Iterators

In [133]:
%%cython

cdef class Iterator:
    cdef readonly list data
    cdef readonly int length
    
    def __cinit__(self, data):
        self.data = data
        self.length = len(data)
    
    def __len__(self):
        return len(self.data)
    
    def __iter__(self):
        cdef int i
        for i in range(self.length):
            yield self.data[i]
    
    cpdef printIter(self):
        for value in self:
            print(value)

In [134]:
it = Iterator([1,2,3])

In [135]:
for value in it: print(value)

1
2
3


In [136]:
it.printIter()

1
2
3
